<a href="https://colab.research.google.com/github/tomifarall/Orga-Datos-Tp2/blob/master/feature_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import re as re

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score


import tensorflow as tf
from tensorflow import keras
from numpy import loadtxt
from tensorflow.keras.layers import Dense, Dropout, Flatten

import tensorflow.keras.backend as K

In [ ]:
#ESTO NO LO EJECUTEN SI UTILIZAN UNA VERSION DE PC , YO LO HAGO PARA PODER USAR EL TRAIN DE MI DRIVE,Y USAR EL GITHUB ONLINE
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1BDGyvJqONz4gk2Y0BK1Ag1DA9yujd44e'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('data_uni_loca_1.csv')

In [ ]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '18Ab39T1hPgTBv58oXdVk0NfBE36Fp21D'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_uni_loca_1.csv')


In [ ]:
#Cargamos los datos

train_data = pd.read_csv("data_uni_loca_1.csv")
test_data = pd.read_csv("test_uni_loca_1.csv")

In [ ]:
train_data.head()

,Unnamed: 0,id,keyword,location,text,target,clean_text
0,0,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale market ablaze
1,1,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt
2,2,50,ablaze,Africa,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...
3,3,52,ablaze,Philadelphia,Crying out for more! Set me ablaze,0,cry set ablaze
4,4,53,ablaze,Florida,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze


In [ ]:
train_data=train_data.drop( columns='Unnamed: 0')
train_data=train_data.drop( columns='clean_text')
test_data=test_data.drop( columns='Unnamed: 0')
test_data=test_data.drop( columns='clean_text')

In [ ]:
#Corregimos algunos nulos del set de train

train_data['keyword'].fillna('no keyword', inplace = True) 
train_data['keyword'] = train_data['keyword'].str.replace('%20', ' ')
train_data['location'].fillna('no location', inplace = True)
train_data.drop(['id'],1,inplace = True)

In [ ]:
#Corregimos algunos nulos del set de test
test_data['keyword'].fillna('no keyword', inplace = True)
test_data['keyword'] = train_data['keyword'].str.replace('%20', ' ')
test_data['location'].fillna('no location', inplace = True)

In [ ]:
#FUNCIONES UTILES

import nltk
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

def only_letters(tweet):
    tweet = re.sub(r'http\S*', '', tweet)
    tweet = re.sub(r'[^a-z\s]', '', tweet)
    return tweet

stop_words=set(stopwords.words("english"))

def filter_stopwords(tokenized_text):
    filtered_words=[]
    for w in tokenized_text:
        if w not in stop_words:
            filtered_words.append(w)
    return filtered_words

lemmatizer = WordNetLemmatizer()

def lemmatize_tweet(tweet):
    lemmatized_words = []
    for word in tweet:
        lemmatized_words.append(lemmatizer.lemmatize(word))
    return lemmatized_words

def transform_to_text(tweet_words):
    return " ".join(tweet_words)

custom_sent_tokenizer = PunktSentenceTokenizer()

def process_content(sentence):
    tokenized = custom_sent_tokenizer.tokenize(sentence)
    words_tagged = []
    for i in tokenized:
        words = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(words)
        for word in tagged:
            words_tagged.append(word)
            
    return words_tagged

#Cleaning text

train_data['clean_text'] = train_data['text'].str.lower()
test_data['clean_text'] = test_data['text'].str.lower()

train_data['clean_text'] = train_data['clean_text'].apply(only_letters)    
test_data['clean_text'] = test_data['clean_text'].apply(only_letters)   

#Tokenización

train_data['clean_text'] = train_data['clean_text'].apply(word_tokenize)
test_data['clean_text'] = test_data['clean_text'].apply(word_tokenize)

#Remove stopwords

train_data['clean_text'] = train_data['clean_text'].apply(filter_stopwords) 
test_data['clean_text'] = test_data['clean_text'].apply(filter_stopwords)

#Lemmatization                                                                       

train_data['clean_text'] = train_data['clean_text'].apply(lemmatize_tweet)
test_data['clean_text'] = test_data['clean_text'].apply(lemmatize_tweet)

#Transform to text
train_data['clean_text'] = train_data['clean_text'].apply(transform_to_text)
test_data['clean_text'] = test_data['clean_text'].apply(transform_to_text)

#Part of speech tagging

train_data['tagged_text']= train_data['text'].apply(process_content)
train_data['tagged_clean_text']= train_data['clean_text'].apply(process_content)

test_data['tagged_text']= test_data['text'].apply(process_content)
test_data['tagged_clean_text']= test_data['clean_text'].apply(process_content)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
def count_Nouns(list):
    nouns = 0
    for word,tag in list:
        if tag[0] == 'N':
            nouns += 1
        else:
            continue
    return nouns

def count_Adjetives(list):
    adjetives = 0
    for word,tag in list:
        if tag[0] == 'J':
            adjetives += 1
        else:
            continue
    return adjetives

def count_Verbs(list):
    verbs = 0
    for word,tag in list:
        if tag[0] == 'V':
            verbs += 1
        else:
            continue
    return verbs 


# word_count
train_data['word_count'] = train_data['text'].apply(lambda x: len(str(x).split()))
test_data['word_count'] = test_data['text'].apply(lambda x: len(str(x).split()))

# unique_word_count
train_data['unique_word_count'] = train_data['text'].apply(lambda x: len(set(str(x).split())))
test_data['unique_word_count'] = test_data['text'].apply(lambda x: len(set(str(x).split())))

# stop_word_count
train_data['stop_word_count'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
test_data['stop_word_count'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

# url_count
train_data['url_count'] = train_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))
test_data['url_count'] = test_data['text'].apply(lambda x: len([w for w in str(x).lower().split() if 'http' in w or 'https' in w]))

# mean_word_length
train_data['mean_word_length'] = train_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_data['mean_word_length'] = test_data['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# length
train_data['tweet_length'] = train_data['text'].apply(lambda x: len(str(x)))
test_data['tweet_length'] = test_data['text'].apply(lambda x: len(str(x)))

# punctuation_count
train_data['punctuation_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
test_data['punctuation_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# hashtag_count
train_data['hashtag_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c == '#']))
test_data['hashtag_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c == '#']))

# mention_count
train_data['mention_count'] = train_data['text'].apply(lambda x: len([c for c in str(x) if c == '@']))
test_data['mention_count'] = test_data['text'].apply(lambda x: len([c for c in str(x) if c == '@']))

#noun_count
train_data['noun_count'] = train_data['tagged_text'].apply(count_Nouns)
test_data['noun_count'] = test_data['tagged_text'].apply(count_Nouns)

#verb_count
train_data['verb_count'] = train_data['tagged_text'].apply(count_Verbs)
test_data['verb_count'] = test_data['tagged_text'].apply(count_Verbs)

#adjetives_count
train_data['adjetives_count'] = train_data['tagged_text'].apply(count_Adjetives)
test_data['adjetives_count'] = test_data['tagged_text'].apply(count_Adjetives)

In [ ]:
train_data.head()

,keyword,location,text,target,clean_text,tagged_text,tagged_clean_text,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,tweet_length,punctuation_count,hashtag_count,mention_count,noun_count,verb_count,adjetives_count
0,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale market ablaze,"[(@, NN), (bbcmtd, JJ), (Wholesale, NNP), (Mar...","[(bbcmtd, JJ), (wholesale, JJ), (market, NN), ...",5,5,0,1,10.200000,55,6,0,1,5,1,1
1,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt,"[(We, PRP), (always, RB), (try, VBP), (to, TO)...","[(always, RB), (try, VB), (bring, VBG), (heavy...",10,10,3,1,5.800000,67,8,2,0,5,2,0
2,ablaze,Africa,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...,"[(#, #), (AFRICANBAZE, NNP), (:, :), (Breaking...","[(africanbaze, NN), (breaking, VBG), (newsnige...",9,9,1,1,8.222222,82,9,1,0,7,2,1
3,ablaze,Philadelphia,Crying out for more! Set me ablaze,0,cry set ablaze,"[(Crying, VBG), (out, RP), (for, IN), (more, J...","[(cry, NN), (set, NN), (ablaze, NN)]",7,7,3,0,4.000000,34,1,0,0,1,2,1
4,ablaze,Florida,On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze,"[(On, IN), (plus, CC), (side, JJ), (LOOK, NNP)...","[(plus, CC), (side, JJ), (look, NN), (sky, NN)...",13,13,5,1,4.923077,76,5,0,0,11,0,1


In [ ]:
test_data.head()

,id,keyword,location,text,clean_text,tagged_text,tagged_clean_text,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,tweet_length,punctuation_count,hashtag_count,mention_count,noun_count,verb_count,adjetives_count
0,46,ablaze,Florida,Birmingham Wholesale Market is ablaze BBC News...,birmingham wholesale market ablaze bbc news fi...,"[(Birmingham, NNP), (Wholesale, NNP), (Market,...","[(birmingham, JJ), (wholesale, JJ), (market, N...",16,14,3,1,6.562500,120,7,0,0,10,2,2
1,47,ablaze,Niall's place | SAF 12 SQUAD |,@sunkxssedharry will you wear shorts for race ...,sunkxssedharry wear short race ablaze,"[(@, NN), (sunkxssedharry, NN), (will, MD), (y...","[(sunkxssedharry, NN), (wear, WRB), (short, JJ...",9,9,3,0,5.111111,54,2,0,1,5,1,0
2,51,ablaze,Nigeria,#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriag...,previouslyondoyintv toke makinwas marriage cri...,"[(#, #), (PreviouslyOnDoyinTv, NN), (:, :), (T...","[(previouslyondoyintv, NN), (toke, NN), (makin...",10,10,0,1,10.000000,109,10,1,0,10,0,1
3,58,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,check nsfw,"[(Check, VB), (these, DT), (out, IN), (:, :), ...","[(check, NN), (nsfw, NN)]",8,8,1,4,13.375000,114,22,1,0,5,1,4
4,60,ablaze,"Los Angeles, Califnordia",PSA: IÛªm splitting my personalities.\n\n?? t...,psa im splitting personality techie follow abl...,"[(PSA, NN), (:, :), (IÛªm, NNP), (splitting, ...","[(psa, NN), (im, NN), (splitting, VBG), (perso...",13,11,1,0,6.230769,94,9,0,2,8,3,0


## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(min_df = 2, max_df = 0.5, ngram_range= (1,2))

#Corremos el algoritmo para TRAIN set 
texts = train_data['clean_text']
features = tf_idf.fit_transform(texts)
feature_words = tf_idf.get_feature_names()
df_tf_idf = pd.DataFrame(data = features.todense(), columns = tf_idf.get_feature_names())
df_tf_idf.shape


(7613, 10911)

In [ ]:
train_data[df_tf_idf.columns]=df_tf_idf


In [ ]:
#Corremos el algoritmo para TEST set 
texts_test = test_data['clean_text']

features_test = tf_idf.transform(texts_test)

feature_words_test = tf_idf.get_feature_names()

df_tf_idf_test = pd.DataFrame(data = features_test.todense(), columns = tf_idf.get_feature_names())
df_tf_idf_test.shape

test_data[df_tf_idf_test.columns]=df_tf_idf_test


In [ ]:
test_data

,id,keyword,location,text,clean_text,tagged_text,tagged_clean_text,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,tweet_length,punctuation_count,hashtag_count,mention_count,noun_count,verb_count,adjetives_count,aa,ab,aba,aba woman,abandon,abandoned,abandoned aircraft,abbott,abbswinston,abbswinston zionist,abc,abc news,abc online,abcnews,abcnews obama,abe,abia,ability,ablaze,ablaze amp,ablaze california,...,youth,youth explosion,youth saved,youtube,youtube channel,youtube military,youtube playlist,youtube video,youve,youve home,ypres,ypres road,yr,yr ago,yr everyone,yr old,yugvani,yyc,yyc abstorm,yyc yycstorm,yycstorm,zakbagans,zarry,zayn,zaynmalik,zero,zimbabwe,zionism,zionist,zionist terrorist,zippednews,zipper,zipper bag,zombie,zombie apocalypse,zone,zone coming,zone dont,zouma,zouma flattened
0,0.0,ablaze,0.0,0.0,birmingham wholesale market ablaze bbc news fi...,"[(Birmingham, NNP), (Wholesale, NNP), (Market,...","[(birmingham, JJ), (wholesale, JJ), (market, N...",16,14,3,1,6.562500,120,7,0,0,10,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.179223,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,ablaze,0.0,0.0,sunkxssedharry wear short race ablaze,"[(@, NN), (sunkxssedharry, NN), (will, MD), (y...","[(sunkxssedharry, NN), (wear, WRB), (short, JJ...",9,9,3,0,5.111111,54,2,0,1,5,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428460,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,ablaze,0.0,0.0,previouslyondoyintv toke makinwas marriage cri...,"[(#, #), (PreviouslyOnDoyinTv, NN), (:, :), (T...","[(previouslyondoyintv, NN), (toke, NN), (makin...",10,10,0,1,10.000000,109,10,1,0,10,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447245,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,ablaze,0.0,0.0,check nsfw,"[(Check, VB), (these, DT), (out, IN), (:, :), ...","[(check, NN), (nsfw, NN)]",8,8,1,4,13.375000,114,22,1,0,5,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,ablaze,0.0,0.0,psa im splitting personality techie follow abl...,"[(PSA, NN), (:, :), (IÛªm, NNP), (splitting, ...","[(psa, NN), (im, NN), (splitting, VBG), (perso...",13,11,1,0,6.230769,94,9,0,2,8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.357133,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3258,0.0,mass murder,0.0,0.0,conference attendee blue line airport derailed...,"[(To, TO), (conference, NN), (attendees, NNS),...","[(conference, NN), (attendee, NN), (blue, JJ),...",24,23,10,0,4.416667,129,4,0,0,7,5,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3259,0.0,mass murder,0.0,0.0,death toll issuicide car bombing ypg position ...,"[(The, DT), (death, NN), (toll, NN), (in, IN),..

In [ ]:
df = train_data.drop(['tagged_text','tagged_clean_text','clean_text','text','keyword','location'], axis=1)

In [ ]:
df

,target,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,tweet_length,punctuation_count,hashtag_count,mention_count,noun_count,verb_count,adjetives_count,aa,ab,aba,aba woman,abandon,abandoned,abandoned aircraft,abbott,abbswinston,abbswinston zionist,abc,abc news,abc online,abcnews,abcnews obama,abe,abia,ability,ablaze,ablaze amp,ablaze california,able,abomb,abomination,abomination maketh,abortion,absolute,...,youth,youth explosion,youth saved,youtube,youtube channel,youtube military,youtube playlist,youtube video,youve,youve home,ypres,ypres road,yr,yr ago,yr everyone,yr old,yugvani,yyc,yyc abstorm,yyc yycstorm,yycstorm,zakbagans,zarry,zayn,zaynmalik,zero,zimbabwe,zionism,zionist,zionist terrorist,zippednews,zipper,zipper bag,zombie,zombie apocalypse,zone,zone coming,zone dont,zouma,zouma flattened
0,0.0,5,5,0,1,10.200000,55,6,0,1,5,1,1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.376739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,10,10,3,1,5.800000,67,8,2,0,5,2,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,9,9,1,1,8.222222,82,9,1,0,7,2,1,0.0,0.0,0.433626,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.394085,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,7,7,3,0,4.000000,34,1,0,0,1,2,1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.474729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,13,13,5,1,4.923077,76,5,0,0,11,0,1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.354690,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.0,26,25,16,0,3.423077,114,1,0,0,4,5,2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7609,0.0,20,18,1,1,5.100000,121,11,0,0,14,0,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7610,0.0,11,11,2,1,6.636364,83,5,0,0,6,1,2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7611,0.0,8,8,2,1,7.250000,65,11,0,0,8,1,0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [ ]:
test = test_data.drop(['tagged_text','tagged_clean_text','clean_text','text','keyword','location'], axis=1)
test.head()

,id,word_count,unique_word_count,stop_word_count,url_count,mean_word_length,tweet_length,punctuation_count,hashtag_count,mention_count,noun_count,verb_count,adjetives_count,aa,ab,aba,aba woman,abandon,abandoned,abandoned aircraft,abbott,abbswinston,abbswinston zionist,abc,abc news,abc online,abcnews,abcnews obama,abe,abia,ability,ablaze,ablaze amp,ablaze california,able,abomb,abomination,abomination maketh,abortion,absolute,...,youth,youth explosion,youth saved,youtube,youtube channel,youtube military,youtube playlist,youtube video,youve,youve home,ypres,ypres road,yr,yr ago,yr everyone,yr old,yugvani,yyc,yyc abstorm,yyc yycstorm,yycstorm,zakbagans,zarry,zayn,zaynmalik,zero,zimbabwe,zionism,zionist,zionist terrorist,zippednews,zipper,zipper bag,zombie,zombie apocalypse,zone,zone coming,zone dont,zouma,zouma flattened
0,0.0,16,14,3,1,6.562500,120,7,0,0,10,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.179223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,9,9,3,0,5.111111,54,2,0,1,5,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.428460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,10,10,0,1,10.000000,109,10,1,0,10,0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.447245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,8,8,1,4,13.375000,114,22,1,0,5,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,13,11,1,0,6.230769,94,9,0,2,8,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.357133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
